In [2]:
#to download counterfact, you can use terminal command:
# curl -o counterfact.json https://rome.baulab.info/data/dsets/counterfact.json

import torch
import pandas as pd

#df = pd.read_pickle("/home/ubuntu/hypernetwork-editor/assets/data/wikipedia_df.pt")
#df.keys()
#Yields:

# Index(['title', 'first_sentences', 'second_sentences', 'third_sentences',
    #    'first_sentence_length', 'second_sentence_length',
    #    'third_sentence_length', 'tokenized_first_sentence',
    #    'tokenized_next_50_tokens'],
    #   dtype='object')

# We will now try to roughly match these up with the Counterfact dataset
path = '../assets/data/counterfact.json'
import json
#load the json file

with open(path, 'r') as f:
    data = json.load(f)

data[0].keys()

dict_keys(['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts'])

In [3]:
data[0]

{'case_id': 0,
 'pararel_idx': 2796,
 'requested_rewrite': {'prompt': 'The mother tongue of {} is',
  'relation_id': 'P103',
  'target_new': {'str': 'English', 'id': 'Q1860'},
  'target_true': {'str': 'French', 'id': 'Q150'},
  'subject': 'Danielle Darrieux'},
 'paraphrase_prompts': ['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native',
  'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'],
 'neighborhood_prompts': ['The mother tongue of Léon Blum is',
  'The native language of Montesquieu is',
  'François Bayrou, a native',
  'The native language of Raymond Barre is',
  'Michel Rocard is a native speaker of',
  'Jacques Chaban-Delmas is a native speaker of',
  'The native language of François Bayrou is',
  'Maurice Genevoix, speaker of',
  'The mother tongue of François Bayrou is',
  'Melchior de Vogüé, speaker of'],
 'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
  'The mother tongue

In [2]:
#check: what datatype is the wikipedia df?
#it is a pandas.core.frame.DataFrame ; so we will try to emulate that

In [4]:
#Write a function which continues a string with gpt2
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move model to the specified device

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
model.to(device)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


Using device: cuda


In [5]:
import pandas as pd

# Function to process your list of dictionaries into a pandas DataFrame
def process_data(data):
    # Initialize an empty list to hold the processed rows
    processed_data = []
    
    # Loop through each entry in the data
    for entry in data:
        # Format the 'instruction' by substituting the subject into the prompt and appending the 'target_new'
        instruction = entry['requested_rewrite']['prompt'].format(entry['requested_rewrite']['subject']) + " " + entry['requested_rewrite']['target_new']['str'] + ". "
        instruction_ids = tokenizer.encode(instruction, return_tensors='pt')

        # Choose one example from 'generation_prompts' (first one in this case)
        generation_example = entry['generation_prompts'][0]
        partial_generation_ids = tokenizer.encode(generation_example, return_tensors='pt')

        # Append the processed data to the list
        processed_data.append({'instruction': instruction, 'instruction_ids': instruction_ids,
                                'generation_example': generation_example, 'partial_generation_ids': partial_generation_ids})
    
    # Convert the list of processed data into a DataFrame
    df = pd.DataFrame(processed_data)
    
    return df

# # Example usage with your data (assuming 'data' is your list of dictionaries)
# data = [
#     {'case_id': 0,
#      'pararel_idx': 2796,
#      'requested_rewrite': {'prompt': 'The mother tongue of {} is',
#                            'relation_id': 'P103',
#                            'target_new': {'str': 'English', 'id': 'Q1860'},
#                            'target_true': {'str': 'French', 'id': 'Q150'},
#                            'subject': 'Danielle Darrieux'},
#      'generation_prompts': ["Danielle Darrieux's mother tongue is",
#                             "Where Danielle Darrieux is from, people speak the language of"]
#     }
#     # Add more dictionaries as needed
# ]

# Process the data
dfc = process_data(data)

# Display or use the DataFrame as needed
#print(dfc)


In [ ]:
from torch.nn.functional import pad
def process_batches(df, batch_size=200):
    # List to store outputs
    generated_ids = []
    generated_texts = []
    
    # Process dataframe in batches
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        
        # Concatenate tensors for each pair in the batch
        concatenated_tensors = [torch.cat([inst.squeeze(0).to(device), gen.squeeze(0).to(device)], dim=0).to(device) for inst, gen in zip(batch['instruction_ids'], batch['partial_generation_ids'])]
        
        # Calculate the maximum length of the tensors in the batch
        max_length = max(t.size(0) for t in concatenated_tensors)
        
        # Manually pad tensors on the left
        padded_tensors = [pad(t, (max_length - t.size(0), 0), value=tokenizer.pad_token_id) for t in concatenated_tensors]
        
        # Stack tensors into a batch and set to cuda
        padded_tensor = torch.stack(padded_tensors, dim=0)

        # Generate responses from the model for the whole batch
        responses = model.generate(padded_tensor, max_length=padded_tensor.size(1) + 50, pad_token_id=tokenizer.pad_token_id)
        
        # Extract only the generated part by slicing the tensor
        full_generated_ids = [response[(padded_tensor.size(1) - gen.shape[1]): (padded_tensor.size(1) - gen.shape[1] + 50)] for response, gen in zip(responses, batch['partial_generation_ids'])]
        
        # Batch decode the generated part of the tensor responses
        generated_text_batch = tokenizer.batch_decode(full_generated_ids, skip_special_tokens=True)

        generated_ids.extend(full_generated_ids)
        generated_texts.extend(generated_text_batch)
        print([i,len(df) / batch_size])

    return [generated_ids, generated_texts]

# Example usage
# Let's assume 'df' is your DataFrame
results = process_batches(dfc)

In [27]:
len(results[0])

21919

In [28]:
#Now rename `instruction_ids` to `tokenized_first_sentence` ; and set `tokenized_next_50_tokens` to results[0]

dfc['tokenized_first_sentence'] = dfc['instruction_ids']
dfc['tokenized_next_50_tokens'] = [result.cpu() for result in results[0]]


In [39]:
tokenizer.decode(dfc["tokenized_next_50_tokens"][10])

'The trademark of BBC One is owned by Sega.\n\nThe trademark of BBC One is owned by Sega. The trademark of BBC One is owned by Sega. The trademark of BBC One is owned by Sega. The trademark of BBC One is owned by'